In [19]:
import pandas as pd

## Connecting to TrueFx

In [20]:
import urllib.request

In [21]:
webUrl = urllib.request.urlopen('https://webrates.truefx.com/rates/connect.html')

HTTPError: HTTP Error 403: Forbidden

In [ ]:
requestUrl = "https://webrates.truefx.com/rates/connect.html"

# Data set configuration

In [ ]:
JanuaryData = pd.read_csv ('../HistoricalData/EurUsd/EURUSD-2023-01.csv')
FebruaryData = pd.read_csv('../HistoricalData/EurUsd/EURUSD-2023-02.csv')

In [ ]:
JanuaryData

In [ ]:
JanuaryData.columns=['EUR/USD','timestamp','price','volume']
FebruaryData.columns=['EUR/USD','timestamp','price','volume']

In [ ]:
JanuaryData

In [ ]:
# JanuaryData.set_index('timestamp',inplace=True)
# FebruaryData.set_index('timestamp',inplace=True)

In [ ]:
JanuaryData['timestamp'] = pd.to_datetime(JanuaryData.timestamp,format='%Y-%m-%d %H:%M:%S.%f')



In [ ]:
FebruaryData['timestamp'] = pd.to_datetime(FebruaryData.timestamp, format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
FebruaryData

In [ ]:
JanuaryData

In [ ]:
appended_data = JanuaryData.append(FebruaryData)

In [ ]:
appended_data

# Converting to currency data

In [ ]:
resampledFrame = appended_data

In [ ]:
resampledFrame

In [ ]:
# resampledFrame.set_index('timestamp',inplace=True)

In [ ]:
# Resample tick data to a higher frequency (e.g., 1 minute) using forward-fill method
df_resampled = resampledFrame.resample('5Sec')

In [ ]:
# Calculate open, high, low, close, and volume
df_resampled['open'] = df_resampled['price']
df_resampled['high'] = df_resampled['price'].rolling(window='1T').max()
df_resampled['low'] = df_resampled['price'].rolling(window='1T').min()
df_resampled['close'] = df_resampled['price']
df_resampled['volume'] = df_resampled['quantity']

In [ ]:
df_resampled

# testing

In [ ]:
CurData

In [ ]:
CurData.columns=['EUR/USD','timestamp','price','volume']

In [ ]:
CurData

In [ ]:
CurData.set_index('timestamp',inplace=True)

In [ ]:
CurData

In [ ]:
allNullVal = pd.DataFrame()

In [ ]:
allNullVal = CurData.isnull()

In [ ]:
nullRows = allNullVal.isnull().sum()
nullRows

In [ ]:
CurData

In [ ]:
# # Example timestamp
# # Convert to DatetimeIndex
# datetime_index = pd.to_datetime(CurData.index, format='%Y%m%d %H:%M:%S.%f')
#
# print('DatetimeIndex:', datetime_index)

In [ ]:
CurData

## Resample to making in timeFrames(1sec,5sec...)

In [ ]:
# Resample tick data into OHLC intervals
ohlc_data = CurData.resample('5Min')
volume_data = CurData['volume'].resample('5Min').sum()

In [ ]:
volume_data

In [ ]:
countNull = (volume_data == 0).sum()

In [ ]:
countNull

In [ ]:
sampler = CurData.index.ohlc()

In [ ]:
# Flatten column names
ohlc_data.columns = ['_'.join(col) for col in ohlc_data.columns]
ohlc_data.drop(columns=['price'], inplace=True)

In [ ]:
# Merge OHLC and volume data
merged_data = pd.concat([ohlc_data, volume_data], axis=1)

# Backtesting config

In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG

ModuleNotFoundError: No module named 'backtesting'

In [ ]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

In [ ]:
cvs_data_default = GOOG
GOOG

In [ ]:
import backtesting

In [ ]:

# Define your trading strategy
class moving_average_crossover(Strategy):
    params = (('short_window', 20), ('long_window', 50))

    def __init__(self):
        self.short_ma = bt.ind.SMA(self.data, period=self.params.short_window)
        self.long_ma = bt.ind.SMA(self.data, period=self.params.long_window)
        self.crossover = bt.ind.CrossOver(self.short_ma, self.long_ma)

    def next(self):
        if not self.position:  # if not in a position
            if self.crossover > 0:  # if short MA crosses above long MA
                self.buy()  # generate a buy signal
        elif self.crossover < 0:  # if short MA crosses below long MA
            self.close()  # generate a sell signal

# Load historical data
data = pd.read_csv('dt/EurUsd/EURUSD-2023-01.csv')

# Create a backtest object

bt = Backtest(GOOG, moving_average_crossover,
              cash=10000, commission=.002,
              exclusive_orders=True)


In [ ]:
output = bt.run()
bt.plot()

In [ ]:
# api = Metatrader(host="172.17.0.2",tz_local=True)

In [ ]:
# api = Metatrader()

In [ ]:
# accountInfo = api.accountInfo()
# accountInfo

In [ ]:
# api = Metatrader(host="145.93.120.219" ,tz_local=True)

# Data Visualization

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
matplotlib.pyplot.plot(JanuaryData.timestamp.head(2000),JanuaryData.price.head(2000))

In [ ]:
matplotlib.pyplot.plot(JanuaryData.timestamp.head(20),JanuaryData.price.head(20))